In [1]:
# !rm -rf diffusers
# !git clone https://github.com/huggingface/diffusers
# !cd diffusers && pip install -e .
!pip install -r requirements_sdxl.txt 


  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-5mvnkded
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-5mvnkded
  Resolved https://github.com/huggingface/diffusers to commit d67eba0f3188a23d12c3e64cae19689b87881295
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 17.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 37.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
from accelerate.utils import write_basic_config
write_basic_config()


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [4]:
MODEL_NAME="stabilityai/stable-diffusion-xl-base-1.0"
INSTANCE_DIR="../data/phil_1024"

STEPS = 1000
SCHEDULER = "cosine"
LR = 3e-6

OUTPUT_DIR=f"{INSTANCE_DIR.split('/')[-1]}_{STEPS}_{SCHEDULER}_{LR}"

!accelerate launch ./train_dreambooth_lora_sdxl.py \
  --seed=0 \
  --pretrained_model_name_or_path={MODEL_NAME}  \
  --pretrained_vae_model_name_or_path=madebyollin/sdxl-vae-fp16-fix \
  --instance_data_dir={INSTANCE_DIR} \
  --output_dir={OUTPUT_DIR} \
  --mixed_precision="bf16" \
  --instance_prompt="a photo of Philippe Saade man" \
  --resolution=1024 \
  --train_batch_size=1 \
  --learning_rate={LR} \
  --report_to="wandb" \
  --lr_scheduler={SCHEDULER} \
  --lr_warmup_steps=0 \
  --max_train_steps={STEPS} \
  --validation_prompt="beautiful Philippe Saade man photography, 8k, highly detailed, 8k" \
  --validation_epochs=9 \
  --seed="0"  \
  --push_to_hub \
  --num_validation_images=2 \
  --enable_xformers_memory_efficient_attention
  # --train_text_encoder \


08/13/2023 16:56:53 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: bf16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'clip_sample_range', 'variance_type', 'dynamic_thresholding_ratio', 'thresholding'} was not found in config. Values will be initialized to default values.
wandb: Currently logged in as: philippe-henri-saade (kollai). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /root/sd-xl-experiments/lora/wandb/run-20230813_165923-9js9ap4b
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run easy-lion-20
wandb: ⭐️ View project at https:

In [3]:
!pip install --upgrade huggingface_hub

  Obtaining dependency information for huggingface_hub from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.8 MB/s eta 0:00:00a 0:00:01
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.13.4
    Uninstalling huggingface-hub-0.13.4:
      Successfully uninstalled huggingface-hub-0.13.4


In [4]:
import torch
torch.cuda.empty_cache()

In [4]:
import huggingface_hub

ModuleNotFoundError: No module named 'huggingface_hub'

In [1]:
from huggingface_hub.repocard import RepoCard
from diffusers import DiffusionPipeline
import torch

lora_model_id = "phil-lora-2"
card = RepoCard.load(lora_model_id)
base_model_id = card.data.to_dict()["base_model"]

pipe = DiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.load_lora_weights(lora_model_id)
image = pipe("A picture of a sks man in a suit", num_inference_steps=25).images[0]
image.save("sks_dog.png")

ModuleNotFoundError: No module named 'huggingface_hub'

In [10]:
from huggingface_hub import notebook_login


ModuleNotFoundError: No module named 'huggingface_hub'

In [6]:
a = 4

In [10]:
from huggingface_hub import upload_folder
import os

def save_model_card(
    repo_id: str, images=None, base_model=str, train_text_encoder=False, prompt=str, repo_folder=None, vae_path=None
):
    img_str = ""
    for i, image in enumerate(images):
        image.save(os.path.join(repo_folder, f"image_{i}.png"))
        img_str += f"![img_{i}](./image_{i}.png)\n"

    yaml = f"""
---
license: openrail++
base_model: {base_model}
instance_prompt: {prompt}
tags:
- stable-diffusion-xl
- stable-diffusion-xl-diffusers
- text-to-image
- diffusers
- lora
inference: true
---
    """
    model_card = f"""
# LoRA DreamBooth - {repo_id}

These are LoRA adaption weights for {base_model}. The weights were trained on {prompt} using [DreamBooth](https://dreambooth.github.io/). You can find some example images in the following. \n
{img_str}

LoRA for the text encoder was enabled: {train_text_encoder}.

Special VAE used for training: {vae_path}.
"""
    with open(os.path.join(repo_folder, "README.md"), "w") as f:
        f.write(yaml + model_card)


save_model_card(
    "PhilSad/phil_1024_750_cosine_0.0001",
    base_model=MODEL_NAME,
    train_text_encoder=True,
    prompt="a photo of sks man",
    repo_folder=OUTPUT_DIR,
    vae_path="adebyollin/sdxl-vae-fp16-fix",
    images=[]
)
upload_folder(
    repo_id="PhilSad/phil_1024_750_cosine_0.0001",
    folder_path=OUTPUT_DIR,
    commit_message="End of training",
    ignore_patterns=["step_*", "epoch_*"],
)

pytorch_lora_weights.bin:   0%|          | 0.00/30.2M [00:00<?, ?B/s]

random_states_0.pkl:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

pytorch_lora_weights.bin:   0%|          | 0.00/30.2M [00:00<?, ?B/s]

optimizer.bin:   0%|          | 0.00/60.3M [00:00<?, ?B/s]

scheduler.bin:   0%|          | 0.00/563 [00:00<?, ?B/s]

'https://huggingface.co/PhilSad/phil_1024_750_cosine_0.0001/tree/main/'